In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, hinge_loss
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn import metrics

In [2]:
data = pd.read_csv("letter-recognition.data", delimiter = ',') 
data.columns =['Label',	'1',	'2',	'3',	'4',	'5',	'6',	'7',	'8',	'9',	'10',	'11',	'12',	'13',	'14',	'15',	'16']
print(data[:5])
indexing = {'A':0, 'B':1, 'C':2, 'D':3, 'E':4, 'F':5, 'G':6, 'H':7, 'I':8, 'J':9, 'K':10, 'L':11, 'M':12, 'N':13, 'O':14, 'P':15, 'Q':16, 'R':17, 'S':18, 'T':19, 'U':20, 'V':21, 'W':22, 'X':23, 'Y':24, 'Z':25}

  Label  1   2  3  4  5   6  7  8  9  10  11  12  13  14  15  16
0     I  5  12  3  7  2  10  5  5  4  13   3   9   2   8   4  10
1     D  4  11  6  8  6  10  6  2  6  10   3   7   3   7   3   9
2     N  7  11  6  6  3   5  9  4  6   4   4  10   6  10   2   8
3     G  2   1  3  1  1   8  6  6  6   6   5   9   1   7   5  10
4     S  4  11  5  8  3   8  8  6  9   5   6   6   0   8   9   7


In [3]:
def k_time_markov_sampling_SVM(Train,n,k,q,n2,Kernel):

  # step 1: draw randomly N sample
  Random_N_sample=Train.sample(n)

  #Train SVMC using differnt Kernel
  X_Random_N_sample=Random_N_sample.drop('Label',axis=1)
  Y_Random_N_sample=Random_N_sample['Label']

  if Kernel=="linear":
    Model_Sample=SVC(kernel = Kernel)
  if Kernel=="rbf":
    Model_Sample=SVC(kernel = Kernel,random_state=0)
  if Kernel=="sigmoid":
    Model_Sample=SVC(kernel = Kernel)
  if Kernel=="poly":
    Model_Sample=SVC(kernel = Kernel,degree = 2)

  # f0
  Model_Sample.fit(X_Random_N_sample,Y_Random_N_sample)

  #step 3
  N=[]
  for i in range(k):
    N.append(0)
    t=1  

    #draw randomly sample zt

    zt=Train.sample()
    N[i]=N[i]+1

    Markov_sample=[]   # for storing the accepted zstar

    #step 4
    cnt=0
    while True:

      zstar=Train.sample()

      #calculation of alpha or acceptance probablity
      zt_x=zt.drop('Label', axis=1)
      zt_y=zt['Label']
      zt_decison = Model_Sample.decision_function(zt_x) 
      zt_pre = max(zt_decison[0]) 

      zstar_x=zstar.drop('Label', axis=1)
      zstar_y=zstar['Label']
      zstar_decison = Model_Sample.decision_function(zstar_x)  
      zstar_pre = max(zstar_decison[0]) 

      alpha = np.exp(np.log2(abs(zstar_pre/zt_pre)))

      #step 5

      if round(alpha,2)==1:
        Markov_sample.append(zstar)
        cnt=0
      elif alpha > 1:
        Markov_sample.append(zstar)
        cnt=0
      elif cnt==n2:
        Markov_sample.append(zstar)
        cnt=0
      else:
        cnt=cnt+1
      
      N[i]=N[i]+1
      zt=zstar

      if N[i]<n:
        continue
      else :
        break

    #Step 6

    Markov_sample = pd.concat(Markov_sample)
    #Train N Markov  
    X_Markov_sample=Markov_sample.drop('Label',axis=1)
    Y_Markov_sample=Markov_sample['Label']

    if Kernel=="linear":
      Markov_Model_Sample=SVC(kernel = Kernel)
    if Kernel=="rbf":
      Markov_Model_Sample=SVC(kernel = Kernel,random_state=0)
    if Kernel=="sigmoid":
      Markov_Model_Sample=SVC(kernel = Kernel)
    if Kernel=="poly":
      Markov_Model_Sample=SVC(kernel = Kernel,degree = 2)


    Markov_Model_Sample.fit(X_Markov_sample,Y_Markov_sample)

  #step 7
  return Markov_Model_Sample

In [4]:
len = round(data.shape[0]*0.8)
train, test = data[:len], data[len:]

In [5]:
test_x = test.drop('Label', axis=1)
test_y = test['Label']

In [6]:
K_time_markov_model = k_time_markov_sampling_SVM(train, 100, 2, 1.2, 10, "linear")
K_time_markov_model_prediction = K_time_markov_model.predict(test_x)
# print(test_y)
# print(" ")
# print(K_time_markov_model_prediction)
print(classification_report(test_y, K_time_markov_model_prediction),"\n")
linear_kernel= metrics.accuracy_score(test_y, K_time_markov_model_prediction)
print("accuracy", metrics.accuracy_score(test_y, K_time_markov_model_prediction), "\n")

              precision    recall  f1-score   support

           A       0.58      0.88      0.70       156
           B       0.19      0.35      0.25       136
           C       0.55      0.63      0.58       142
           D       0.26      0.20      0.23       167
           E       0.56      0.63      0.59       152
           F       0.43      0.60      0.50       153
           G       0.31      0.16      0.21       164
           H       0.14      0.17      0.15       151
           I       0.78      0.35      0.48       165
           J       0.60      0.44      0.51       148
           K       0.30      0.47      0.37       146
           L       0.91      0.78      0.84       157
           M       0.35      0.51      0.42       144
           N       0.29      0.12      0.17       166
           O       0.29      0.50      0.37       139
           P       0.32      0.33      0.32       168
           Q       0.94      0.18      0.30       168
           R       0.47    

In [7]:
K_time_markov_model = k_time_markov_sampling_SVM(train, 100, 2, 1.2, 10, "rbf")
K_time_markov_model_prediction = K_time_markov_model.predict(test_x)
# print(test_y)
# print(" ")
# print(K_time_markov_model_prediction)
print(classification_report(test_y, K_time_markov_model_prediction),"\n")
rbf_kernel= metrics.accuracy_score(test_y, K_time_markov_model_prediction)
print("accuracy", metrics.accuracy_score(test_y, K_time_markov_model_prediction), "\n")

              precision    recall  f1-score   support

           A       0.97      0.58      0.72       156
           B       0.16      0.10      0.12       136
           C       0.33      0.63      0.44       142
           D       0.36      0.06      0.10       167
           E       0.10      0.64      0.18       152
           F       0.00      0.00      0.00       153
           G       0.00      0.00      0.00       164
           H       0.00      0.00      0.00       151
           I       0.00      0.00      0.00       165
           J       0.76      0.61      0.67       148
           K       0.12      0.24      0.16       146
           L       0.57      0.72      0.64       157
           M       0.00      0.00      0.00       144
           N       0.35      0.04      0.08       166
           O       0.09      0.48      0.16       139
           P       0.24      0.90      0.37       168
           Q       0.00      0.00      0.00       168
           R       0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
K_time_markov_model = k_time_markov_sampling_SVM(train, 100, 2, 1.2, 10, "sigmoid")
K_time_markov_model_prediction = K_time_markov_model.predict(test_x)
# print(test_y)
# print(" ")
# print(K_time_markov_model_prediction)
print(classification_report(test_y, K_time_markov_model_prediction),"\n")
sigmoid_kernel= metrics.accuracy_score(test_y, K_time_markov_model_prediction)
print("accuracy", metrics.accuracy_score(test_y, K_time_markov_model_prediction), "\n")

              precision    recall  f1-score   support

           A       0.00      0.00      0.00       156
           B       0.00      0.00      0.00       136
           C       0.00      0.00      0.00       142
           D       0.00      0.00      0.00       167
           E       0.00      0.00      0.00       152
           F       0.00      0.00      0.00       153
           G       0.00      0.00      0.00       164
           H       0.00      0.00      0.00       151
           I       0.00      0.00      0.00       165
           J       0.04      1.00      0.07       148
           K       0.00      0.00      0.00       146
           L       0.00      0.00      0.00       157
           M       0.00      0.00      0.00       144
           N       0.00      0.00      0.00       166
           O       0.00      0.00      0.00       139
           P       0.00      0.00      0.00       168
           Q       0.00      0.00      0.00       168
           R       0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
K_time_markov_model = k_time_markov_sampling_SVM(train, 100, 2, 1.2, 10, "poly")
K_time_markov_model_prediction = K_time_markov_model.predict(test_x)
# print(test_y)
# print(" ")
# print(K_time_markov_model_prediction)
print(classification_report(test_y, K_time_markov_model_prediction),"\n")
polynomial_kernel= metrics.accuracy_score(test_y, K_time_markov_model_prediction)
print("accuracy", metrics.accuracy_score(test_y, K_time_markov_model_prediction), "\n")

              precision    recall  f1-score   support

           A       0.68      0.88      0.77       156
           B       0.22      0.15      0.18       136
           C       0.85      0.59      0.70       142
           D       0.37      0.69      0.48       167
           E       0.56      0.20      0.30       152
           F       0.48      0.66      0.56       153
           G       0.21      0.74      0.33       164
           H       0.21      0.62      0.31       151
           I       0.48      0.68      0.57       165
           J       0.98      0.42      0.59       148
           K       0.19      0.07      0.10       146
           L       0.00      0.00      0.00       157
           M       0.88      0.70      0.78       144
           N       0.64      0.70      0.67       166
           O       0.00      0.00      0.00       139
           P       0.71      0.49      0.58       168
           Q       0.71      0.36      0.48       168
           R       0.63    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
print("Accuracy\n")
print("linear_kernel     : ",linear_kernel*100)
print("rbf_kernel        : ",rbf_kernel*100)
print("sigmoid_kernel    : ",sigmoid_kernel*100)
print("polynomial_kernel : ",polynomial_kernel*100)

Accuracy

linear_kernel     :  42.575
rbf_kernel        :  24.775
sigmoid_kernel    :  3.6999999999999997
polynomial_kernel :  45.2


In [12]:
print("Misclassification rate\n")
print("linear_kernel     : ",1-linear_kernel)
print("rbf_kernel        : ",1-rbf_kernel)
print("sigmoid_kernel    : ",1-sigmoid_kernel)
print("polynomial_kernel : ",1-polynomial_kernel)

Misclassification rate

linear_kernel     :  0.5742499999999999
rbf_kernel        :  0.75225
sigmoid_kernel    :  0.963
polynomial_kernel :  0.548
